In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from category_encoders import CountEncoder
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from IPython.display import Audio
import matplotlib.pyplot as plt
from fastFM import als
from category_encoders import CatBoostEncoder

In [ ]:
from IPython.display import Audio
# Cargo los archivos
sound_file = '/kaggle/input/pingsound/PING - Sound effect.mp3'
def ping():
    display(Audio(sound_file, autoplay=True))

In [ ]:
es_para_submit = True
agregar_estimacion_preciom2 = True
if es_para_submit: 
    TAMANIO_TRAIN = 240000
    TAMANIO_TEST = 60000
else:
    TAMANIO_TRAIN = 190000
    TAMANIO_TEST = 50000

In [ ]:
test = pd.read_csv("../input/tp2datostrain/test.csv")
display(test.shape)
train = pd.read_csv("../input/tp2datostrain/train.csv")
display(train.shape)

In [ ]:
ciudades = list(train["ciudad"].unique())
tipos_de_propiedad = list(train["tipodepropiedad"].unique())
provincia = list(train["provincia"].unique())
categorias  = [provincia, tipos_de_propiedad, ciudades]

In [ ]:
df = train

In [ ]:
features = [#cOLUMNAS
            'idzona',
            'lat', 'lng',
            "metrostotales", "metroscubiertos", 
            'garages', 'banos',
            'gimnasio','centroscomercialescercanos','usosmultiples','piscina','escuelascercanas',
            'antiguedad',
            'habitaciones',
            'provincia', 'tipodepropiedad', 'ciudad',
           ]

In [ ]:
transformador_categorias = make_pipeline(SimpleImputer(strategy='most_frequent'),
                                         OneHotEncoder(handle_unknown='ignore', categories = categorias))
transformador_numeros = make_pipeline(SimpleImputer())

In [ ]:
def obtenerFeaturesCategoricos(features):
    l = []
    for feature in features:
        if not np.issubdtype(df[feature].dtype, np.number):
            l.append(feature)
    return l

num_features = list(df[features].select_dtypes(include=[np.number]).columns)

cat_features = list(obtenerFeaturesCategoricos(features))
print ("Features numericos:", num_features, '\nFeatures categoricos: ', cat_features)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, num_features),
        ('cat', transformador_categorias, cat_features)
    ])

In [ ]:
X = train[features].copy()
y = train["precio"].copy()
train_x, test_x, train_y, test_y = train_test_split(X, y,train_size=0.8, test_size=0.2)
from fastFM import als
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)
x_procesada = preprocessor.fit_transform(train_x)
x_test_procesada = preprocessor.fit_transform(test_x)

fm.fit(x_procesada, train_y)
y_pred = fm.predict(x_test_procesada)

In [ ]:
ping()
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_pred, test_y)

In [ ]:
X = train[features].copy()
y = train["precio"].copy()
train_x, test_x, train_y, test_y = train_test_split(X, y,train_size=0.8, test_size=0.2)
from fastFM import sgd
fm2 = sgd.FMRegression(n_iter=1000, init_stdev=0.1, rank=2, step_size=0.00000000001)
x_procesada = preprocessor.fit_transform(train_x)
x_test_procesada = preprocessor.fit_transform(test_x)

fm2.fit(x_procesada, train_y)
y_pred2 = fm2.predict(x_test_procesada)

In [ ]:
ping()
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_pred2, test_y)

PAra predecir los precios con los features normales no sirve de mucho FM

Pruebo un poco con el preprocesamiento y features mas copados

In [ ]:
dfCheto = pd.concat([train,test], sort=False)
dfCheto['precio'] = SimpleImputer(strategy='median').fit_transform(dfCheto[['precio']])
df = dfCheto.copy()

In [ ]:
df.provincia.fillna('provinciaVacia', inplace = True)
aux2 = df[['ciudad', 'provincia']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby(['provincia']).agg(lambda x:x.provincia.value_counts().index[0]).reset_index().set_index('ciudad')
ciuds = aux2.T.to_dict('records').copy()
ciuds = ciuds[0]
df.ciudad.fillna(df.provincia.map(ciuds), inplace = True)
df.ciudad.fillna('ciudadVacia', inplace=True)

df.tipodepropiedad.fillna('tipoVacio', inplace=True)

aux2 = df[['idzona', 'ciudad']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby('ciudad').agg({'idzona':'median'})
ids = aux2.T.to_dict('records').copy()
ids = ids[0]
df.idzona.fillna(df.ciudad.map(ids), inplace = True)
df['idzona'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['idzona']])

#Adicionales
aux = df.copy()
aux.descripcion.fillna('',inplace=True)
aux['jardin'] = (aux.descripcion.str.contains('jardin') | aux.descripcion.str.contains('jardín')).astype(int)
aux['vigilancia'] = (aux.descripcion.str.contains('vigilancia') | aux.descripcion.str.contains('seguridad')).astype(int) 
aux['buenaUbicacion'] = (aux.descripcion.str.contains('ubicacion') | aux.descripcion.str.contains('ubicación')).astype(int)
aux['lavadero'] = (aux.descripcion.str.contains('lavadero') | aux.descripcion.str.contains('lavado')).astype(int)
aux['jacuzzi'] = (aux.descripcion.str.contains('jacuzzi')).astype(int)
def buscar_barrioCerrado(string):
    return int(('barrio cerrado' in string) or ('urbanizacion cerrada' in string) or ('urbanización cerrada' in string)
    or ('barrio residencial' in string) or ('country' in string) or ('barrio privado' in string) or ('colonia privada' in string))
aux['barrio_cerrado'] = aux.descripcion.map(lambda x:buscar_barrioCerrado(str(x)))

valor_adicionales = {'gimnasio':4,'usosmultiples':2,'piscina':5,'escuelascercanas':3,'centroscomercialescercanos':3,
                     'buenaUbicacion':2,'vigilancia': 5,'jardin':4,'jacuzzi':3,'lavadero':0.5,'barrio_cerrado':10}
            
for columna in aux.columns:
    if columna in valor_adicionales:
        aux[columna] = aux[columna].map(lambda x: x*valor_adicionales[columna])

aux['adicionales'] = aux[valor_adicionales.keys()].sum(axis=1)
df['adicionales'] = aux['adicionales']


aux = df[['lat', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lat':'mean'})
lats = aux.T.to_dict('records').copy()
lats = lats[0]
df.lat.fillna(df.ciudad.map(lats), inplace = True)
df['lat'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lat']])

aux = df[['lng', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lng':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.lng.fillna(df.ciudad.map(lngs), inplace = True)
df['lng'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lng']])

aux = df[['tipodepropiedad', 'habitaciones']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'habitaciones':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.habitaciones.fillna(df.tipodepropiedad.map(habts), inplace = True)
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])

aux = df[['tipodepropiedad', 'garages']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'garages':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.garages.fillna(df.tipodepropiedad.map(habts), inplace = True)
df.garages.fillna(0, inplace=True)

df.drop(columns=['direccion'], inplace=True)


aux = df[['antiguedad', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'antiguedad':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.antiguedad.fillna(df.ciudad.map(lngs), inplace = True)
df['antiguedad'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['antiguedad']])

aux = df[['banos', 'tipodepropiedad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'banos':'median'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.banos.fillna(df.tipodepropiedad.map(lngs), inplace = True)
df.banos.fillna(1, inplace=True)

aux = df.copy()
aux.metrostotales.fillna(0, inplace=True)
aux.metroscubiertos.fillna(0, inplace =True)
aux['metrostotalesCorregidos'] = aux[['metrostotales', 'metroscubiertos']].max(axis=1)
aux['metroscubiertosCorregidos'] = aux[['metrostotales', 'metroscubiertos']].min(axis=1)
df['metrostotales'] = aux['metrostotalesCorregidos']
df['metroscubiertos'] = aux['metroscubiertosCorregidos']



df['fecha'] = pd.to_datetime(df['fecha'])
df['dia'] = df.fecha.dt.day
df['mes'] = df.fecha.dt.month
df['anio'] = df.fecha.dt.year
df.drop(columns=['fecha'], inplace=True)

df['publicacionesPorCiudad'] = CountEncoder().fit_transform(df['ciudad'])
df['publicacionesPorProvincia'] = CountEncoder().fit_transform(df['provincia'])

df['publicacionesporZonaYAnio'] = df.groupby(['idzona','anio'])['antiguedad'].transform('count')

aux = df[['mes', 'anio']].copy()
aux['cantidad'] =  1
aux['cantidad'] = aux.groupby(['mes', 'anio']).transform('sum')
df['publicacionesEnMes'] = aux['cantidad']

aux = df[['metrostotales', 'tipodepropiedad']].copy()
aux['metrosPromedioPorTipo'] = aux.groupby('tipodepropiedad').transform('mean')
df['metrosPromedioPorTipo'] = aux['metrosPromedioPorTipo']



aux = df[['metrostotales', 'ciudad']].copy()
aux['metrosPromedioPorCiudad'] = aux.groupby('ciudad').transform('mean')
df['metrosPromedioPorCiudad'] = aux['metrosPromedioPorCiudad']



In [ ]:
transformador_categorias = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

transfomador_numero = Pipeline(steps=[('a', SimpleImputer(strategy = 'median'))])



preprocessor = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero, ['antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng','adicionales',
       'dia', 'mes', 'anio','publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesporZonaYAnio',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad']),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])



In [ ]:
model = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)

In [ ]:
entrenamiento = df.head(TAMANIO_TRAIN).copy()
prueba = df.tail(TAMANIO_TEST).copy()
prueba.drop(columns=['precio'], inplace=True)
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)


In [ ]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [ ]:
my_pipeline.fit(train_x, train_y)

In [ ]:
preds = my_pipeline.predict(train_x)

In [ ]:
from sklearn.metrics import mean_absolute_error
ping()
mean_absolute_error(preds, train_y)

Vemos que aún con los features utilizados en unas buenas estimaciones que tuvimos FM no da buen resultado.

OPeero podríamos usarlo para clasificar o predecir texto

### Trato de generar un feature que sea una aproximacion del precio en funcion del texto

In [ ]:
aux = dfCheto[["titulo", "descripcion", "direccion", "precio"]].copy()
aux.fillna("",inplace = True)
aux["texto"] = aux["titulo"].astype(str) + " " + aux["descripcion"].astype(str) +" "+ aux["direccion"].astype(str)

In [ ]:
import string
tabla = {}
for c in string.punctuation + "¿¡":
    tabla[ord(c)] = None
for c in string.ascii_uppercase:
    tabla[ord(c)] = c.lower()
def sacar_puntuacion_y_minusculizar(s):
    return s.translate(tabla)

aux["texto"] = aux["texto"].apply(sacar_puntuacion_y_minusculizar)

from nltk.corpus import stopwords
palabras_vacias = stopwords.words('spanish')
import unidecode
for i in range(len(palabras_vacias)):
    palabras_vacias[i] = unidecode.unidecode(palabras_vacias[i])

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', stop_words=palabras_vacias)

transformador_texto = Pipeline(steps=[('tf-idf', vectorizer)])
procesador_texto = ColumnTransformer(transformers=[
        ('text', transformador_texto, ['texto'])])

In [ ]:
model = als.FMRegression(n_iter=250, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)
pipeline_texto = Pipeline(steps=[('preprocessor', procesador_texto),
                              ('model', model)
                             ])


In [ ]:
entrenamiento = aux.head(TAMANIO_TRAIN).copy()
train_x = entrenamiento["texto"].copy()
train_y = entrenamiento.precio.copy()

texto_procesado = transformador_texto.fit_transform(train_x)

In [ ]:
texto_procesado.shape

In [ ]:
model.fit(texto_procesado, train_y)

In [ ]:
preds = model.predict(texto_procesado)
ping()

In [ ]:
from sklearn.metrics import mean_absolute_error
ping()
mean_absolute_error(preds, train_y)

In [ ]:
prueba = aux.tail(TAMANIO_TEST).copy()
#prueba.drop(columns=['precio'], inplace=True)
test_y = prueba["precio"].copy()
texto_procesado_test = transformador_texto.transform(prueba["texto"])
texto_procesado_test.shape

In [ ]:
preds2 = model.predict(texto_procesado_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
ping()
mean_absolute_error(preds2, test_y)

In [ ]:
ping()
preds2 = np.absolute(preds2)

In [ ]:
res = pd.DataFrame(preds2, index=test.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# RMSLE=1.0249284784393988 ?


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(res)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
entrenamiento = aux.head(TAMANIO_TRAIN).copy()
train_x = entrenamiento["texto"].copy()
train_y = entrenamiento.precio.copy()

X_train, X_test, y_train, y_test = train_test_split(
   train_x , train_y, test_size=0.33, random_state=42)

X_train_trans = transformador_texto.transform(X_train)
X_test_trans = transformador_texto.transform(X_test)

n_iter = 10
step_size = 2
l2_reg_w = 0
l2_reg_V = 0

params =100000
fm = als.FMRegression(n_iter=0, init_stdev = 0.28, l2_reg=params, rank=2)
# Allocates and initalizes the model parameter.
fm.fit(X_train_trans, y_train)

rmse_train = []
rmse_test = []
r2_score_train = []
r2_score_test = []
for i in range(1, n_iter):
    print(fm.get_params())
    fm.fit(X_train_trans, y_train, n_more_iter=step_size)
    params *= 1.10
    fm.set_params(l2_reg=params, l2_reg_w=params, l2_reg_V=params)
    
    y_pred_test = fm.predict(X_test_trans)
    y_pred_train = fm.predict(X_train_trans)

    rmse_train.append(mean_absolute_error(y_pred_train, y_train))
    rmse_test.append(mean_absolute_error(y_pred_test, y_test))

    r2_score_train.append(r2_score(y_pred_train, y_train))
    r2_score_test.append(r2_score(y_pred_test, y_test))


In [ ]:
ping()
x = np.arange(1, n_iter) * step_size
fig, axes = plt.subplots(ncols=2, figsize=(15, 4))
axes[0].plot(x, rmse_train, label='MAE-train', color='r', ls="--")
axes[0].plot(x, rmse_test, label='MAE-test', color='r')

axes[1].plot(x, r2_score_train, label='R^2-train', color='b', ls="--")
axes[1].plot(x, r2_score_test, label='R^2-test', color='b')
#axes[0].set_yscale('log')
#axes[1].set_yscale('log')

axes[0].set_ylabel('MAE', color='r')
axes[1].set_ylabel('R^2', color='b')
axes[0].legend()
axes[1].legend()

In [ ]:
rmse_test[-1]

In [ ]:
15426355.523708496 5 iter l2_reg=5, rank=8
15199490.841102434 5 iter l2_reg=5, rank=2 #Empeoro
16948064.1257035   5 iter l2_reg=5, rank=1 # Mas peor
10969900.727196898 10 iter l2_reg=5, rank=8 #Mejor
6385817.322031822  10 iter l2_reg=10, rank=8 #Mejor
4593583.346254185  10 iter l2_reg=20, rank=8 #mejoro pero creo que debe overfittear
2446010.0858626226 10 iter l2_reg=200, rank=8 #better
2187496.2726370473 10 iter l2_reg=400, rank=8 #Mejoro pero falta
1901927.550437147  10 iter l2_reg=1000, rank=8 #mejor
1698296.3342013592 10 iter l2_reg=5000, rank=8 #mejor pero luego de 2 iteraciones comenzo a subir
1584523.3392954106 10 iter l2_reg=5000, rank=2 #mejoro extrañamente
1494395.7616398989 10 iter l2_reg=10000, rank=2
1636825.7935355457 10 iter l2_reg=10000, rank=5 #peor
1461863.6166319877 10 iter l2_reg=20000, rank=2 #mejor
1309379.2221132882 10 iter l2_reg=100000, rank=2 #mejor
1552464.7361177704 50 iter l2_reg=10000, rank=2 # empeoro
1486699.8753103984 10 iter init_stdev = 0.3, l2_reg=10000, rank=2 #mejoro
1519634.902997233  10 iter init_stdev = 0.5, l2_reg=10000, rank=2 #empeoro
1552335.8695174404 10 iter init_stdev = 0.01, l2_reg=10000, rank=2 #empeoro
1218251.7260080273 10 iter init_stdev = 0.3, l2_reg=200000, rank=2 #mejoro
1127341.8178689817 10 iter init_stdev = 0.3, l2_reg=500000, rank=2 #mejoro pero me esta cansando
1066328.1431204537 10 iter init_stdev = 0.3, l2_reg=1500000, rank=2 #Basta no lo logro bajar del millon
1073065.6412416825 10 iter init_stdev = 0.25, l2_reg=1500000, rank=3 #empeoro
1066728.843381326 10 iter init_stdev = 0.8, l2_reg=1500000, rank=3 #meh
1042973.0560466803 20 iter init_stdev = 0.3, l2_reg=1300000, rank=2 #mejoro poniendole mas iteraciones
1047046.1476110086 35 iter init_stdev = 0.3, l2_reg=1000000, rank=2 #todaia empeoro
1056651.3892042716 35 iter init_stdev = 0.3, l2_reg=800000, rank=2 #todavia empeoro
1049735.7639088458 60 iter init_stdev = 0.3, l2_reg=800000, rank=2 #sigue sin se mejor que el mejor que tuve
1065502.5001307093 60 iter init_stdev = 0.3, l2_reg=800000, rank=2 #el rank lo unico que hace es empeorar todo
1066526.916931029  60 iter init_stdev = 0.03, l2_reg=800000, rank=3 
1041682.2088397375 10 iter step 5 size 5 init_stdev = 0.28, l2_reg=1000000, rank=2 update 0.75
1041682.2088397375 10 iter step 5 size 5 init_stdev = 0.28, l2_reg=1000000, rank=2 update 0.5

In [ ]:
print(fm.get_params())